In [1]:
import os 
os.listdir('/kaggle/input') 

embeddings_path = "/kaggle/input/embeddings"
print(os.listdir(embeddings_path))
import pandas as pd

speeches_embeddings = pd.read_csv(f"{embeddings_path}/speeches_with_embeddings.csv")
news_embeddings = pd.read_csv(f"{embeddings_path}/news_with_embeddings.csv")


['speeches_with_embeddings.csv', 'speeches_embeddings_sentiment.csv', 'news_embeddings_sentiment.csv', 'all_texts_embeddings.joblib', 'news_with_embeddings.csv']


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

# Convert and verify date columns
news_embeddings['news_date'] = pd.to_datetime(news_embeddings['Date'])
speeches_embeddings['speech_date'] = pd.to_datetime(speeches_embeddings['date'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#Temporal window calculation and expansion
def generate_temporal_pairs(news_df, speeches_df, window_days=3):
    """Generate news-speech pairs within a symmetric temporal window (-3 to +3 days)"""
    pairs = []
    chunk_size = 2000
    news_chunks = np.array_split(news_df, len(news_df) // chunk_size + 1)
    
    for chunk in news_chunks:
        for _, row in chunk.iterrows():
            news_date = row['news_date']
            start_date = news_date - pd.Timedelta(days=window_days) 
            end_date = news_date + pd.Timedelta(days=window_days)  
            
            mask = (speeches_df['speech_date'] >= start_date) & (speeches_df['speech_date'] <= end_date)
            speech_ids = speeches_df[mask].index.tolist()
            pairs.extend([(row.name, s_id) for s_id in speech_ids])
    
    return pd.DataFrame(pairs, columns=['news_id', 'speech_id'])


alignment_df = generate_temporal_pairs(news_embeddings, speeches_embeddings)

#optimized embeddings calculation
def load_embeddings_half(df, col_name):
    embeddings = []
    for i, row in df.iterrows():
        if isinstance(row[col_name], str):
            arr = np.fromstring(row[col_name].strip("[]"), sep=" ", dtype=np.float16)
        else:
            arr = np.array(row[col_name], dtype=np.float16)
        embeddings.append(torch.tensor(arr, device=device).half())
        if i % 1000 == 0: torch.cuda.empty_cache()
    return torch.stack(embeddings)

news_tensor = load_embeddings_half(news_embeddings, 'news_embeddings')
speeches_tensor = load_embeddings_half(speeches_embeddings, 'speech_embeddings')

#Batched cosine similarity computation
def compute_cosine_similarities(pairs_df, news_emb, speech_emb, batch_size=8192):
    news_norm = F.normalize(news_emb, p=2, dim=1)
    speech_norm = F.normalize(speech_emb, p=2, dim=1)
    similarities = []
    for i in range(0, len(pairs_df), batch_size):
        batch = pairs_df.iloc[i:i+batch_size]
        news_batch = news_norm[batch['news_id'].values]
        speech_batch = speech_norm[batch['speech_id'].values]
        similarities.append(F.cosine_similarity(news_batch, speech_batch).cpu().numpy())
        del news_batch, speech_batch
        torch.cuda.empty_cache()
    return np.concatenate(similarities)

alignment_df['cosine_similarity'] = compute_cosine_similarities(alignment_df, news_tensor, speeches_tensor)

#include metadata to the embeddings to track temporal dependencies
def add_temporal_features(pairs_df, news_df, speeches_df):
    pairs_df = pairs_df.merge(
        news_df[['news_date']],
        left_on='news_id',
        right_index=True
    ).merge(
        speeches_df[['speech_date']],
        left_on='speech_id',
        right_index=True
    )
    pairs_df['days_diff'] = (pairs_df['news_date'] - pairs_df['speech_date']).dt.days
    return pairs_df

enriched_df = add_temporal_features(alignment_df, news_embeddings, speeches_embeddings)

#Save data to avoid rerunning everything again 
enriched_df.to_parquet('news_speech_similarities.parquet', engine='pyarrow', compression='zstd')
print("Processing complete. Results saved with columns:", enriched_df.columns.tolist())

Using device: cpu


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
import matplotlib.pyplot as plt
# Load processed data
df = pd.read_parquet('news_speech_similarities.parquet') #We can use this code when we compute everything again, else we have to roead the parquet
#df = pd.read_parquet(SIMILARITIES_PATH) #Piece of code when we have the path defined, otherwise it'll work by loading them again. 
# Convert to datetime and normalize (remove time components)
df['news_date'] = pd.to_datetime(df['news_date']).dt.normalize()
df['year'] = df['news_date'].dt.year

# Create daily aggregates with std dev
daily_agg = df.groupby('news_date')['cosine_similarity'].agg(['mean', 'std']).reset_index()
daily_agg.columns = ['date', 'cosine_similarity', 'std_dev']

# Extend full_dates to include October 2024 explicitly
end_date = pd.to_datetime('2024-10-31')  # Adjust as needed
full_dates = pd.date_range(
    start=daily_agg['date'].min(), 
    end=end_date, 
    freq='D'
)
daily_agg = daily_agg.set_index('date').reindex(full_dates).reset_index().rename(columns={'index': 'date'})

# Calculate bounds
daily_agg['upper_bound'] = daily_agg['cosine_similarity'] + daily_agg['std_dev'].fillna(0)
daily_agg['lower_bound'] = daily_agg['cosine_similarity'] - daily_agg['std_dev'].fillna(0)

# Create monthly aggregates (fill NaN with 0 for plotting)
monthly_agg = daily_agg.set_index('date').resample('M')['cosine_similarity'].mean().fillna(0).reset_index()
monthly_agg['month_label'] = monthly_agg['date'].dt.strftime('%b\n%Y')

# Get unique years present in data
years = daily_agg['date'].dt.year.unique()

# Set up plot
plt.figure(figsize=(18, 8 * len(years)))

for i, year in enumerate(years, 1):
    year_mask = daily_agg['date'].dt.year == year
    yearly_daily = daily_agg[year_mask]
    yearly_monthly = monthly_agg[monthly_agg['date'].dt.year == year]
    
    if yearly_daily.empty:
        continue
    
    ax = plt.subplot(len(years), 1, i)
    
    # Daily plot with variability
    ax.plot(yearly_daily['date'], 
            yearly_daily['cosine_similarity'], 
            color='#2ca02c', 
            linewidth=1.5,
            label='Daily Average')
    
    ax.fill_between(yearly_daily['date'],
                    yearly_daily['upper_bound'],
                    yearly_daily['lower_bound'],
                    color='gray', alpha=0.3, 
                    label='Daily Std Dev')
    
    # Monthly markers (plot even if value is 0)
    ax.scatter(yearly_monthly['date'], 
               yearly_monthly['cosine_similarity'],
               color='darkblue', 
               s=100,
               zorder=5,
               label='Monthly Average')
    
    # Annotate monthly values (skip if 0)
    for _, row in yearly_monthly.iterrows():
        if row['cosine_similarity'] != 0:
            ax.text(row['date'], row['cosine_similarity']+0.02,
                    f"{row['cosine_similarity']:.2f}",
                    ha='center', va='bottom',
                    fontsize=9, color='darkblue')
    
    # Highlight missing days
    missing_mask = yearly_daily['cosine_similarity'].isna()
    ax.fill_between(yearly_daily['date'],
                    yearly_daily['cosine_similarity'].min() - 0.1,
                    yearly_daily['cosine_similarity'].max() + 0.1,
                    where=missing_mask,
                    color='red', alpha=0.1,
                    label='Missing Days')
    
    # Formatting
    ax.set_title(f'{year} Daily/Monthly Speech-News Cosine Similarity', pad=20)
    ax.set_xlabel('')
    ax.set_ylabel('Cosine Similarity', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')
    
    # Set monthly x-ticks
    ax.set_xticks(yearly_monthly['date'])
    ax.set_xticklabels(yearly_monthly['month_label'])
    
    # Set y-axis limits
    y_min = max(daily_agg['cosine_similarity'].min() - 0.1, 0)
    y_max = min(daily_agg['cosine_similarity'].max() + 0.1, 1)
    ax.set_ylim(y_min, y_max)

plt.tight_layout()
plt.show()

/tmp/ipykernel_72/322553216.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_agg = daily_agg.set_index('date').resample('M')['cosine_similarity'].mean().fillna(0).reset_index()


In [4]:
import re
# again extracting top media outlets. Maybe there is a more efficient way to do this. 
def extract_outlet(url):
    patterns = [r"https?://(?:www\.)?([^/.]+)\."]
    match = re.search(patterns[0], str(url), re.IGNORECASE)
    return match.group(1).lower() if match else "unknown"

# Create outlet column in news_embeddings
news_embeddings['outlet'] = news_embeddings['Link'].apply(extract_outlet)

In [5]:
# Now this will work

enriched_df = enriched_df.merge(
    news_embeddings[['outlet']],
    left_on='news_id',
    right_index=True,
    how='left'
)

# Final check
print("\nColumns in enriched_df:", enriched_df.columns.tolist())
print("Sample outlets:", enriched_df['outlet'].unique()[:5])


Columns in enriched_df: ['news_id', 'speech_id', 'cosine_similarity', 'news_date', 'speech_date', 'days_diff', 'outlet']
Sample outlets: ['bbc' 'politica' 'oem' 'eleconomista' 'milenio']


### First regressions

In regression 1, I have to change the variable days_diff into dummies. 
In regression 2, I have to  

In [6]:
import pandas as pd
import statsmodels.api as sm

# 1. Create dummy variables for outlets
dummies_outlet = pd.get_dummies(enriched_df['outlet'], prefix='outlet')

# 2. Filter for top outlets
top_outlets = enriched_df['outlet'].value_counts().nlargest(10).index.tolist()
dummy_outlet_columns = [f'outlet_{outlet}' for outlet in top_outlets if f'outlet_{outlet}' in dummies_outlet.columns]
dummies_outlet = dummies_outlet[dummy_outlet_columns]

# 3. Create dummies for days_diff
enriched_df['days_diff'] = pd.to_numeric(enriched_df['days_diff'], errors='coerce')
dummies_days = pd.get_dummies(enriched_df['days_diff'], prefix='days_diff')
dummies_days = dummies_days.drop(columns='days_diff_0', errors='ignore')  # Drop baseline

# 4. Combine dummies
dummies = pd.concat([dummies_outlet, dummies_days], axis=1)

# 5. Handle missing values
valid_data = enriched_df[['cosine_similarity']].join(dummies).dropna()

# ✅ Ensure all columns are numeric
X = sm.add_constant(valid_data[dummies.columns]).astype(float)
y = valid_data['cosine_similarity'].astype(float)

# 6. Run regression
model = sm.OLS(y, X).fit(cov_type='HC3')

# 7. Output summary
print("\n=== Outlet + Days Diff Fixed Effects Model ===")
print(model.summary())



=== Outlet + Days Diff Fixed Effects Model ===
                            OLS Regression Results                            
Dep. Variable:      cosine_similarity   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                 1.500e+05
Date:                Wed, 16 Apr 2025   Prob (F-statistic):               0.00
Time:                        09:36:01   Log-Likelihood:             2.7810e+07
No. Observations:            24418171   AIC:                        -5.562e+07
Df Residuals:                24418154   BIC:                        -5.562e+07
Df Model:                          16                                         
Covariance Type:                  HC3                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------

## Regression with temporal dependencies (dummies per month and year)¶

In [7]:
import gc
gc.collect()

384

In [8]:
print("Unique days_diff values:", enriched_df['days_diff'].unique())


Unique days_diff values: [-3 -2 -1  0  1  2  3]


In [7]:
import pandas as pd
import statsmodels.formula.api as smf

# --- Data Preparation ---
# Convert to categorical types upfront
enriched_df = enriched_df.astype({
    'outlet': 'category',
    'news_date': 'datetime64[ns]',
    'speech_date': 'datetime64[ns]'
})

# Create temporal features without expanding memory
enriched_df['month'] = enriched_df['news_date'].dt.month.astype('int8')
enriched_df['year'] = enriched_df['news_date'].dt.year.astype('int16')

# Filter to top outlets based on frequency
top_outlets = enriched_df['outlet'].value_counts().nlargest(10).index.tolist()

# --- Minimal changes to recode outlet ---
# First, add 'Other' as a valid category, then recode.
enriched_df['outlet'] = enriched_df['outlet'].cat.add_categories('Other')
enriched_df.loc[~enriched_df['outlet'].isin(top_outlets), 'outlet'] = 'Other'
enriched_df['outlet'] = enriched_df['outlet'].cat.set_categories(top_outlets + ['Other'])

# Downcast numerical columns for efficiency
enriched_df['cosine_similarity'] = pd.to_numeric(enriched_df['cosine_similarity'], downcast='float')
enriched_df['days_diff'] = pd.to_numeric(enriched_df['days_diff'], downcast='integer')

# If days_diff should only be between -3 and +3, filter the data:
enriched_df = enriched_df[enriched_df['days_diff'].between(-3, 3)]

# --- Instead of using the full dataset, take a subsample for estimation ---
# Adjust the sample size as needed (here, 100000 observations are used)
sample_size = 10000000
enriched_sample = enriched_df.sample(n=sample_size, random_state=42)

# --- Model Specification ---
# Treat days_diff as categorical to generate dummy variables (7 dummies max)
formula = """cosine_similarity ~ C(days_diff) +
C(outlet, Treatment('Other')) + 
C(month) + 
C(year)
"""

# Fit model on the subsample with robust standard errors
model = smf.ols(formula, data=enriched_sample, missing='drop').fit(cov_type='HC3')

print(model.summary())


ValueError: new categories must not include old categories: {'Other'}

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# --- Data Preparation ---
# Convert to categorical types upfront
enriched_df = enriched_df.astype({
    'outlet': 'category',
    'news_date': 'datetime64[ns]',
    'speech_date': 'datetime64[ns]'
})

# Create temporal features without expanding memory
enriched_df['month'] = enriched_df['news_date'].dt.month.astype('int8')
enriched_df['year'] = enriched_df['news_date'].dt.year.astype('int16')

# Filter to top outlets based on frequency
top_outlets = enriched_df['outlet'].value_counts().nlargest(10).index.tolist()

# --- Minimal change to solve the error ---
# First add 'Other' as a valid category:
enriched_df['outlet'] = enriched_df['outlet'].cat.add_categories('Other')
# Now recode any outlet not in top_outlets to 'Other'
enriched_df.loc[~enriched_df['outlet'].isin(top_outlets), 'outlet'] = 'Other'
# Optionally, reset the category order:
enriched_df['outlet'] = enriched_df['outlet'].cat.set_categories(top_outlets + ['Other'])

# Downcast numerical columns for efficiency
enriched_df['cosine_similarity'] = pd.to_numeric(enriched_df['cosine_similarity'], downcast='float')
enriched_df['days_diff'] = pd.to_numeric(enriched_df['days_diff'], downcast='integer')

# --- Model Specification ---
# Treat days_diff as categorical to produce dummy variables (should be 7 dummies for -3,...,3)
formula = """cosine_similarity ~ C(days_diff) +
C(outlet, Treatment('Other')) + 
C(month) + 
C(year)
"""

# Fit the model; missing='drop' automatically drops rows with NA values.
model = smf.ols(formula, data=enriched_df, missing='drop').fit(cov_type='HC3')

print(model.summary())


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# --- Data Preparation ---
# Convert to categorical types upfront
enriched_df = enriched_df.astype({
    'outlet': 'category',
    'news_date': 'datetime64[ns]',
    'speech_date': 'datetime64[ns]'
})

# Create temporal features without expanding memory
enriched_df['month'] = enriched_df['news_date'].dt.month.astype('int8')
enriched_df['year'] = enriched_df['news_date'].dt.year.astype('int16')

# Filter to top outlets using category reordering:
top_outlets = enriched_df['outlet'].value_counts().nlargest(10).index.tolist()

# Build a unique list of categories; if 'Other' is not already present, add it.
categories = top_outlets.copy()
if 'Other' not in categories:
    categories.append('Other')

# Set the categories and recode non-top outlets to 'Other'
enriched_df['outlet'] = enriched_df['outlet'].cat.set_categories(categories)
enriched_df.loc[~enriched_df['outlet'].isin(top_outlets), 'outlet'] = 'Other'

# Downcast numerical columns
enriched_df['cosine_similarity'] = pd.to_numeric(
    enriched_df['cosine_similarity'], 
    downcast='float'
)
enriched_df['days_diff'] = pd.to_numeric(
    enriched_df['days_diff'], 
    downcast='integer'
)

# --- Model Specification ---
# Use formula API with categorical variables
formula = """cosine_similarity ~ days_diff +
C(outlet, Treatment('Other')) + 
C(month) + 
C(year)
"""

# Fit model with reduced memory footprint
model = smf.ols(
    formula, 
    data=enriched_df,
    missing='drop'  # Automatically drops NA rows
).fit(cov_type='HC3')

print(model.summary())